<a href="https://www.kaggle.com/code/mdnadeemakhtar/book-recommendation-system?scriptVersionId=143902299" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
book = pd.read_csv('/kaggle/input/book-recommendation-dataset/Books.csv')
rating = pd.read_csv('/kaggle/input/book-recommendation-dataset/Ratings.csv')
user = pd.read_csv('/kaggle/input/book-recommendation-dataset/Users.csv')

In [ ]:
book.head(2)

In [ ]:
book[book['Book-Title']=='Classical Mythology'].values

In [ ]:
rating.head(2)

In [ ]:
user.head(2)

In [ ]:
print(book.shape)
print(rating.shape)
print(user.shape)

In [ ]:
book.isnull().sum()

In [ ]:
book = book.dropna()

In [ ]:
book.isnull().sum()

In [ ]:
rating.isnull().sum()

In [ ]:
user.isnull().sum()

In [ ]:
user = user.dropna()

In [ ]:
user.isnull().sum()

In [ ]:
book.duplicated().sum()

In [ ]:
user.duplicated().sum()

In [ ]:
rating.duplicated().sum()

# Popularity based Recommender System

In [ ]:
# Top 50 books based on ratings 

In [ ]:
rating

In [ ]:
book

In [ ]:
rating.merge(book, on='ISBN')

In [ ]:
rating.merge(book, on='ISBN').shape

In [ ]:
ratings_with_book_name = rating.merge(book, on='ISBN')

In [ ]:
ratings_with_book_name.groupby('Book-Title').count()['Book-Rating']

In [ ]:
num_rating_df = ratings_with_book_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_rating'}, inplace=True)
num_rating_df

In [ ]:
avg_rating_df = ratings_with_book_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'}, inplace=True)
avg_rating_df

In [ ]:
popular_df = num_rating_df.merge(avg_rating_df, on='Book-Title')
popular_df

In [ ]:
popular_df[popular_df['num_rating']>=250]

In [ ]:
popular_df = popular_df[popular_df['num_rating']>=250].sort_values('avg_rating',ascending=False).head(50)

In [ ]:
popular_df

In [ ]:
popular_df.merge(book, on='Book-Title')

In [ ]:
popular_df = popular_df.merge(book, on='Book-Title').drop_duplicates('Book-Title')

In [ ]:
popular_df.shape

In [ ]:
popular_df = popular_df[['Book-Title','Book-Author','Image-URL-M','num_rating','avg_rating']]

In [ ]:
popular_df

# Collaborative Filtering Based Recommendation

In [ ]:
x = ratings_with_book_name.groupby('User-ID').count()['Book-Rating']>200
excellent_users = x[x].index

In [ ]:
filtered_rating = ratings_with_book_name[ratings_with_book_name['User-ID'].isin(excellent_users)]

In [ ]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [ ]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [ ]:
final_ratings

In [ ]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
pt

In [ ]:
pt.fillna(0,inplace=True)

In [ ]:
pt

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(pt)

In [ ]:
similarity.shape

In [ ]:
# It gives sorted values by index
sorted_similarity_by_index = sorted(list(enumerate(similarity[0])),key=lambda x:x[0])
top_10_similarities_by_index = sorted_similarity_by_index[:10]
top_10_similarities_by_index

In [ ]:
# It gives sorted value by similarity and use reverse=True to get similarity in descending order
sorted_similarity_by_value = sorted(list(enumerate(similarity[0])),key=lambda x:x[1],reverse=True)
top_10_similarities_by_value = sorted_similarity_by_value[:10]
top_10_similarities_by_value

In [ ]:
def recommend(book_name):
    #index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity[index])),key=lambda x:x[1],reverse=True)[1:6]
    
    for i in similar_items:                   
        print(pt.index[i[0]])

In [ ]:
recommend('1984')

In [ ]:
recommend('Harry Potter and the Prisoner of Azkaban (Book 3)')

In [ ]:
recommend('The Hobbit : The Enchanting Prelude to The Lord of the Rings')

In [ ]:
recommend('Life of Pi')

In [ ]:
recommend('Where the Red Fern Grows')

In [ ]:
import pickle

In [ ]:
pickle.dump(final_ratings,open('final_ratings.pkl','wb'))

In [ ]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(book,open('book.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))